In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
cuda = torch.cuda.is_available()
import numpy as np
import collections
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from skimage import io
import torch.nn as nn
import torch.hub
import torchvision.models
import torchvision.models.resnet
import torchvision

In [2]:
from torchvision import models

In [3]:
print(torchvision.__version__)

0.4.2


In [4]:
dir(models)

['AlexNet',
 'DenseNet',
 'GoogLeNet',
 'Inception3',
 'MNASNet',
 'MobileNetV2',
 'ResNet',
 'ShuffleNetV2',
 'SqueezeNet',
 'VGG',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_utils',
 'alexnet',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'detection',
 'googlenet',
 'inception',
 'inception_v3',
 'mnasnet',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet',
 'mobilenet_v2',
 'resnet',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'segmentation',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'shufflenet_v2_x1_5',
 'shufflenet_v2_x2_0',
 'shufflenetv2',
 'squeezenet',
 'squeezenet1_0',
 'squeezenet1_1',
 'utils',
 'vgg',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19',
 'vgg19_bn',
 'video',
 'wide_resnet101_2',
 'wide_resnet50_2']

In [7]:
import torchvision
from PIL import Image

#981
train_size= 589
val_size= 196
test_size = 196

In [8]:
import csv

In [9]:
data_map = {}

In [10]:
val_data_map = {}

In [11]:
with open("training.csv", newline = '') as csvfile:
    reader = csv.reader(csvfile, delimiter = ',')
    for row in reader:
        data_map[row[0]] = row[6]

In [12]:
with open("validation.csv", newline = '') as csvfile:
    reader = csv.reader(csvfile, delimiter = ',')
    for row in reader:
        val_data_map[row[0]] = row[6]

In [13]:
len(data_map)

414800

In [14]:
len(val_data_map)

5500

In [15]:
def parse_data(datadir, train_data_map, val_data_map):
    train_img_list = []
    val_img_list = []
    
    for root, directories, filenames in os.walk(datadir):      
        for filename in filenames:
            if filename.endswith('.jpg'):      
                filei = os.path.join(root, filename)
                split_filename = filei.split("/")
                filename = split_filename[1] + "/" + split_filename[2]
                if filename in train_data_map:
                    train_img_list.append(filei)
                elif filename in val_img_list:
                    val_img_list.append(filei)
               
    return train_img_list, val_img_list

In [16]:
train_img_list, val_img_list = parse_data("Manually_Annotated_Images", data_map, val_data_map)

In [17]:
len(train_img_list)

64724

In [28]:
class ImageDataset(Dataset):
    def __init__(self, img_list, data_map):
        self.img_list = img_list
        self.data_map = data_map
        
        
    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, index):
        
        img_file = self.img_list[index]
        img = Image.open(img_file)
        img_h = 224
        img_w = 224
        img = torchvision.transforms.Resize((img_h,img_w))(img)
        img = torchvision.transforms.ToTensor()(img)
        split_filename = img_file.split("/")
        filename = split_filename[1] + "/" + split_filename[2]
        label = self.data_map[filename]
        return img, int(label)

In [29]:
train_data = ImageDataset(train_img_list, data_map)

In [31]:
val_data = ImageDataset(val_img_list, val_data_map)

In [49]:
batch_size = 64
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=8)

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [50]:
#model = torch.hub.load('pytorch/vision:v0.4.2', 'shufflenet_v2_x1_0', pretrained=True)
model = models.shufflenet_v2_x1_0(pretrained = False)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if cuda else "cpu")

In [51]:
def train(model,n_epochs,train_dataloader, test_loader):
    model.train()
    model.to(device)
    train_losses = []
    eval_losses = []
    eval_accs = []
    for epoch in range(n_epochs):
        avg_loss = 0.0
        for batch_num, (feats, labels) in enumerate(train_dataloader):
            feats, labels = feats.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(feats)
            loss = criterion(outputs, labels.long())
            loss.backward()
            
            optimizer.step()
            
            avg_loss += loss.item()
            if batch_num % 50 == 49:
                logger.info('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/50))
 
                avg_loss = 0.0    
        
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
        train_loss, train_accuracy = test_classify_loss(model,train_dataloader)
        test_loss, test_accuracy = test_classify_loss(model,test_loader)
        eval_losses.append(test_loss)
        train_losses.append(train_loss)
        eval_accs.append(test_accuracy)
        logger.info('Epoch: {}\tTrain Loss: {}\tTrain Acc: {}\tTest-Loss: {}\tTest-acc: {:.4f}'.format(epoch+1, train_loss,train_accuracy, test_loss, test_accuracy))
    return train_losses, eval_losses, eval_accs

def test_classify_loss(model, test_loader):
    with torch.no_grad():
        model.eval()
        test_loss = []
        accuracies = 0
        total = 0
        for batch_num, (feats, labels) in enumerate(test_loader):
            feats, labels = feats.to(device), labels.to(device)
            outputs = model(feats)
            _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
            pred_labels = pred_labels.view(-1)
            loss = criterion(outputs, labels.long())
            accuracies += float(torch.sum(torch.eq(pred_labels, labels)).item())
            total+=float(len(labels))
            test_loss.extend([loss.item()]*feats.size()[0])
            torch.cuda.empty_cache()
            del feats
            del labels
    model.train()
    return np.mean(test_loss), accuracies/total

In [52]:
import logging
logging.basicConfig(filename="training_baseline.log" ,
                            filemode="a+")
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [48]:
train_losses, eval_losses, eval_accs = train(model,10, train_dataloader, None)

2019-11-19 20:48:44,191 root         INFO     Epoch: 1	Batch: 50	Avg-Loss: 3.7700
2019-11-19 20:48:44,191 root         INFO     Epoch: 1	Batch: 50	Avg-Loss: 3.7700


KeyboardInterrupt: 